In [45]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate

class ExchangeRateFetcher:
    @staticmethod
    def get_exchange_rate_for_date(date_str, base_currency="USD", symbols=None):
        url = f"https://api.frankfurter.app/{date_str}"
        params = {"from": base_currency}
        if symbols:
            params["to"] = ",".join(symbols)
        response = requests.get(url, params=params)
        return response.json().get("rates", {}) if response.status_code == 200 else {}

class DataProcessor:
    @staticmethod
    def estimate_statistics(exchange_data):
        df = pd.DataFrame.from_dict(exchange_data, orient="index").sort_index()
        df.dropna(inplace=True)
        if df.shape[0] < 2:
            avg_vector = df.mean() if not df.empty else pd.Series(1, index=target_currencies)
            cov_matrix = np.identity(len(target_currencies)) * 0.01
            return avg_vector, cov_matrix
        return df.mean(), df.cov()

    @staticmethod
    def generate_exchange_rate_samples(avg_vector, cov_matrix, num_samples=100):
        mvn_distribution = multivariate_normal(mean=avg_vector, cov=cov_matrix, allow_singular=True)
        sample_df = pd.DataFrame(mvn_distribution.rvs(size=num_samples, random_state=1), columns=target_currencies)
        sample_df["USD"] = 1
        sample_df.fillna(1, inplace=True)
        sample_df.replace([np.inf, -np.inf], 1, inplace=True)
        return sample_df

class DataSetup:
    @staticmethod
    def setup_demand():
        return pd.DataFrame({
            'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
            'd_h': [7, 15, 5, 7, 3, 18],
            'd_r': [7, 12, 3, 8, 3, 17],
        }).set_index('from')

    @staticmethod
    def setup_capacities():
        return pd.DataFrame({
            'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
            'cap': [18, 45, 18, 10, 30, 22],
        }).set_index('plant')

    @staticmethod
    def setup_production_costs():
        return pd.DataFrame({
            'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
            'fc_p': [20, 45, 14, 13, 30, 23],
            'fc_h': [5, 13, 3, 4, 6, 5],
            'fc_r': [5, 13, 3, 4, 6, 5],
            'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
            'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
            'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
            'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
        }).set_index('plant')

    @staticmethod
    def setup_transportation_costs():
        return pd.DataFrame({
            'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
            'LatinAmerica': [0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
            'Europe': [0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
            'AsiaWoJapan': [0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
            'Japan': [0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
            'Mexico': [0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
            'U.S.': [0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
        }).set_index('from')

    @staticmethod
    def setup_duties():
        return pd.DataFrame({
            'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
            'duty': [0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
        }).set_index('from')

    @staticmethod
    def setup_plant_currency_mapping():
        return {
            'Brazil': 'BRL',
            'Germany': 'EUR',
            'India': 'INR',
            'Japan': 'JPY',
            'Mexico': 'MXN',
            'U.S.': 'USD'
        }

class OptimizationModel:
    def __init__(self, demand, caps, pcosts, tcosts, duties, plant_currency, exchange_rates):
        self.demand = demand
        self.caps = caps
        self.pcosts = pcosts
        self.tcosts = tcosts
        self.duties = duties
        self.plant_currency = plant_currency
        self.exchange_rates = exchange_rates
        self.plants = caps.index.tolist()
        self.regions = demand.index.tolist()
        self.products = ['H', 'R']

    def build_model(self):
        model = Model("Network_Optimization")

        # Decision Variables
        open_plant = model.addVars(self.plants, vtype=GRB.BINARY, name="open_plant")
        produce_h = model.addVars(self.plants, vtype=GRB.BINARY, name="produce_h")
        produce_r = model.addVars(self.plants, vtype=GRB.BINARY, name="produce_r")
        flow = model.addVars(self.plants, self.regions, self.products, name="flow")

        # Objective Function: Minimize Total Cost
        total_cost = quicksum(
            self.pcosts.loc[p, 'fc_p'] * open_plant[p] +
            self.pcosts.loc[p, 'fc_h'] * produce_h[p] +
            self.pcosts.loc[p, 'fc_r'] * produce_r[p] +
            quicksum(
                (self.pcosts.loc[p, 'rm_h'] + self.pcosts.loc[p, 'pc_h']) * flow[p, r, 'H'] +
                (self.pcosts.loc[p, 'rm_r'] + self.pcosts.loc[p, 'pc_r']) * flow[p, r, 'R'] +
                self.tcosts.loc[p, r] * (flow[p, r, 'H'] + flow[p, r, 'R']) * (1 + self.duties.loc[r, 'duty']) *
                self.exchange_rates[self.plant_currency[p]]
                for r in self.regions
            )
            for p in self.plants
        )
        model.setObjective(total_cost, GRB.MINIMIZE)

        # Constraints
        # Capacity Constraints
        model.addConstrs(
            (quicksum(flow[p, r, pr] for r in self.regions for pr in self.products) <= self.caps.loc[p, 'cap'] * open_plant[p]
             for p in self.plants), name="capacity")

        # Demand Constraints
        model.addConstrs(
            (quicksum(flow[p, r, 'H'] for p in self.plants) >= self.demand.loc[r, 'd_h'] for r in self.regions),
            name="demand_h")
        model.addConstrs(
            (quicksum(flow[p, r, 'R'] for p in self.plants) >= self.demand.loc[r, 'd_r'] for r in self.regions),
            name="demand_r")

        # Production Constraints
        model.addConstrs(
            (flow[p, r, 'H'] <= produce_h[p] * self.caps.loc[p, 'cap'] for p in self.plants for r in self.regions),
            name="produce_h")
        model.addConstrs(
            (flow[p, r, 'R'] <= produce_r[p] * self.caps.loc[p, 'cap'] for p in self.plants for r in self.regions),
            name="produce_r")

        return model, open_plant, produce_h, produce_r, flow

    def solve(self):
        model, open_plant, produce_h, produce_r, flow = self.build_model()
        model.optimize()

        if model.status == GRB.OPTIMAL:
            # Extract results
            flow_results = pd.DataFrame(index=self.plants, columns=self.regions)
            for p in self.plants:
                for r in self.regions:
                    flow_results.loc[p, r] = flow[p, r, 'H'].X + flow[p, r, 'R'].X

            strategy_results = pd.DataFrame({
                'Plant': [open_plant[p].X for p in self.plants],
                'H': [produce_h[p].X for p in self.plants],
                'R': [produce_r[p].X for p in self.plants]
            }, index=self.plants)

            return flow_results, strategy_results, model.ObjVal
        else:
            raise Exception("Optimization failed")

# Main Execution
if __name__ == "__main__":
    target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]
    date_list = ["2023-09-30", "2023-10-31", "2023-11-30", "2023-12-31"]

    all_strategies = []

    # Fetch exchange rates
    exchange_data = {date: ExchangeRateFetcher.get_exchange_rate_for_date(date, "USD", target_currencies) for date in date_list}

    # Estimate statistics
    avg_vector, cov_matrix = DataProcessor.estimate_statistics(exchange_data)

    # Generate exchange rate samples
    sample_df = DataProcessor.generate_exchange_rate_samples(avg_vector, cov_matrix, num_samples=100)

    # Setup data
    demand = DataSetup.setup_demand()
    caps = DataSetup.setup_capacities()
    pcosts = DataSetup.setup_production_costs()
    tcosts = DataSetup.setup_transportation_costs()
    duties = DataSetup.setup_duties()
    plant_currency = DataSetup.setup_plant_currency_mapping()

    # Loop through each exchange rate scenario
    for scenario, exchange_rates in sample_df.iterrows():
        print(f"\nScenario {scenario} - Exchange Rates:")
        print(exchange_rates)

        # Solve optimization model
        model = OptimizationModel(demand, caps, pcosts, tcosts, duties, plant_currency, exchange_rates)
        flow_results, strategy_results, min_cost = model.solve()

        # Print results
        print(f"\nScenario {scenario} - HighCal Flow")
        print(tabulate(flow_results, headers='keys', tablefmt='pretty'))

        print(f"\nScenario {scenario} - Strategy")
        print(tabulate(strategy_results, headers='keys', tablefmt='pretty'))

        print(f"\nScenario {scenario} - Minimum Cost: $ {min_cost:.2f}")


Scenario 0 - Exchange Rates:
BRL      4.816552
EUR      0.891939
INR     83.279347
JPY    140.575437
MXN     16.872194
USD      1.000000
Name: 0, dtype: float64
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 90 rows, 90 columns and 294 nonzeros
Model fingerprint: 0x4abdaff0
Variable types: 72 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective 1939.8102667
Presolve time: 0.00s
Presolved: 90 rows, 90 columns, 294 nonzeros
Variable types: 72 continuous, 18 integer (18 binary)

Root relaxation: objective 1.349459e+03, 52 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [52]:
import pandas as pd
import numpy as np
 
# Define the plants
plants = ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.']
 
# Simulated scenario results: frequency of plant opening across different scenarios
# Values represent the proportion of scenarios where the plant is open (1 means always open, 0 means always closed)
scenario_results = {
    'Brazil': 0.6,  # Open in 60% of scenarios
    'Germany': 0.9,  # Open in 90% of scenarios
    'India': 0.7,  # Open in 70% of scenarios
    'Japan': 0.4,  # Open in 40% of scenarios
    'Mexico': 0.8,  # Open in 80% of scenarios
    'U.S.': 1.0  # Always open
}
 
# Define threshold levels for opening/closing
high_threshold = 0.8  # Plants open in more than 80% of scenarios should be kept open
medium_threshold = 0.5  # Plants open between 50-80% should be selectively open
low_threshold = 0.5  # Plants open in less than 50% should be closed
 
# Strategy 1: Conservative Approach (Keep only highly active plants open)
strategy_1 = {p: 1 if scenario_results[p] >= high_threshold else 0 for p in plants}
 
# Strategy 2: Balanced Approach (Keep active and moderately used plants open)
strategy_2 = {p: 1 if scenario_results[p] >= medium_threshold else 0 for p in plants}
 
# Strategy 3: Flexible Approach (Optimize based on cost trade-offs, keeping even some low-frequency plants open)
strategy_3 = {p: 1 if scenario_results[p] >= low_threshold else 0 for p in plants}
 
# Convert to DataFrame for display
strategy_df = pd.DataFrame({
    'Plant': plants,
    'Conservative': [strategy_1[p] for p in plants],
    'Balanced': [strategy_2[p] for p in plants],
    'Flexible': [strategy_3[p] for p in plants]
}).set_index('Plant')
 
# Display the strategies
print(strategy_df)

         Conservative  Balanced  Flexible
Plant                                    
Brazil              0         1         1
Germany             1         1         1
India               0         1         1
Japan               0         0         0
Mexico              1         1         1
U.S.                1         1         1


In [70]:






def compute_fixed_strategy_cost(strategy, exchange_rates):
    fixed_costs = {"Brazil": 20, "Germany": 45, "India": 14, "Japan": 13, "Mexico": 30, "U.S.": 23}
    variable_costs = {"Brazil": 5, "Germany": 13, "India": 3, "Japan": 4, "Mexico": 6, "U.S.": 5}
    currencies = {"Brazil": "BRL", "Germany": "EUR", "India": "INR", "Japan": "JPY", "Mexico": "MXN", "U.S.": "USD"}
 
    total_cost = 0
    for plant, open_status in strategy.items():
        if open_status:
            total_cost += fixed_costs[plant] * exchange_rates[currencies[plant]]
            total_cost += variable_costs[plant] * exchange_rates[currencies[plant]] * 10  # Scaled demand factor
 
    return total_cost
 
# Evaluate each strategy over the exchange rate samples
strategy_costs = {strategy: [] for strategy in fixed_strategies}
 
for i in range(num_samples):
    exchange_rates = exchange_rate_samples.iloc[i]
    for strategy_name, strategy in fixed_strategies.items():
        strategy_costs[strategy_name].append(compute_fixed_strategy_cost(strategy, exchange_rates))
 
# Compute expected cost and cost standard deviation for each strategy
strategy_evaluation = {
    strategy: {"Expected Cost": np.mean(costs), "Cost Std Dev": np.std(costs)}
    for strategy, costs in strategy_costs.items()
}
 
# Convert to DataFrame for display
strategy_evaluation_df = pd.DataFrame(strategy_evaluation)
print(strategy_evaluation_df)

NameError: name 'fixed_strategies' is not defined